In [13]:
%pwd #Đang đứng ở /content

'/content'

In [ ]:
import os
os.chdir('../')

In [ ]:
%pwd

In [16]:
!pip install pi-heif "unstructured[local-inference,pdf]" pdfminer.six pypdf Pillow
!pip install -U langchain langchain-community langchain-core
!pip install -U langchain-text-splitters




In [17]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


def load_pdf_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.pdf",
        loader_cls=UnstructuredPDFLoader
    )
    documents = loader.load()
    return documents

In [18]:
extracted_data = load_pdf_files(data='DataChatbot')

In [3]:
extracted_data

[Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf'}, page_content='BỘ Y TẾ\n\nHƯỚNG DẪN CHẨN ĐOÁN VÀ ĐIỀU TRỊ CÁC BỆNH DA LIỄU (ban hành kèm theo Quyết định số 75/QĐ-BYT ngày 13/01/2015)\n\nHà Nội, 2015\n\n1\n\nBỘ Y TẾ\n\nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc\n\nSố: 75/QĐ-BYT\n\nHà Nội, ngày 13 tháng 01 năm 2015\n\nQUYẾT ĐỊNH Về việc ban hành tài liệu chuyên môn “Hướng dẫn chẩn đoán và điều trị các bệnh da liễu”\n\nBỘ TRƯỞNG BỘ Y TẾ\n\nCăn cứ Luật khám bệnh, chữa bệnh năm 2009;\n\nCăn cứ Nghị định số 63/2012/NĐ-CP ngày 31/8/2012 của Chính Phủ quy\n\nđịnh chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Y tế;\n\nTheo đề nghị của Cục trưởng Cục Quản lý Khám, chữa bệnh,\n\nQUYẾT ĐỊNH:\n\nĐiều 1. Ban hành kèm theo Quyết định này tài liệu chuyên môn “Hướng dẫn\n\nchẩn đoán và điều trị các bệnh da liễu”.\n\nĐiều 2. Tài liệu “Hướng dẫn chẩn đoán và điều trị các bệnh da liễu” ban hành kèm theo Quyết định này được áp dụng tại các

Preprocessing (Tiền xử lý dữ liệu)

In [19]:
import re

def preprocess_data(text):


    #Xóa URL
    text = re.sub(r'(https?://\S+|www\.\S+)', '', text)
    # Xoá các dòng chỉ chứa dấu = hoặc -
    text = re.sub(r'^[=\-]{2,}\s*$', '', text, flags=re.MULTILINE)
    #Xoá các ký tự bảng markdown (|, ---)
    text = re.sub(r'\|.*?\|', '', text)
    # Xoá emoji và ký tự Unicode không cần thiết
    text = re.sub(r'[^\w\s,.!?à-ỹÀ-Ỹ\-–]', '', text)
    # Xoá khoảng trắng thừa và dòng trống
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [6]:
!pip install -U langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


In [20]:
from langchain_core.documents import Document

cleaned_data = []
for doc in extracted_data:
    cleaned_content = preprocess_data(doc.page_content)
    cleaned_doc = Document(
        page_content=cleaned_content,
        metadata=doc.metadata
    )
    cleaned_data.append(cleaned_doc)

In [21]:
cleaned_data

[Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf'}, page_content='BỘ Y TẾ HƯỚNG DẪN CHẨN ĐOÁN VÀ ĐIỀU TRỊ CÁC BỆNH DA LIỄU ban hành kèm theo Quyết định số 75QĐ-BYT ngày 13012015 Hà Nội, 2015 1 BỘ Y TẾ CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc Số 75QĐ-BYT Hà Nội, ngày 13 tháng 01 năm 2015 QUYẾT ĐỊNH Về việc ban hành tài liệu chuyên môn Hướng dẫn chẩn đoán và điều trị các bệnh da liễu BỘ TRƯỞNG BỘ Y TẾ Căn cứ Luật khám bệnh, chữa bệnh năm 2009 Căn cứ Nghị định số 632012NĐ-CP ngày 3182012 của Chính Phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Y tế Theo đề nghị của Cục trưởng Cục Quản lý Khám, chữa bệnh, QUYẾT ĐỊNH Điều 1. Ban hành kèm theo Quyết định này tài liệu chuyên môn Hướng dẫn chẩn đoán và điều trị các bệnh da liễu. Điều 2. Tài liệu Hướng dẫn chẩn đoán và điều trị các bệnh da liễu ban hành kèm theo Quyết định này được áp dụng tại các cơ sở khám bệnh, chữa bệnh. Căn cứ vào tài liệu này và điều kiện cụ th

Chunking

In [23]:
def text_split(cleaned_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    text_chunks = text_splitter.split_documents(cleaned_data)
    return text_chunks


In [24]:
text_chunks = text_split(cleaned_data)
print(f'So luong chunk da tach', len(text_chunks))

So luong chunk da tach 1087


Embeddings vector -> (Vietnamese) 768-dimensional vector space

In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="dangvantuan/vietnamese-embedding"
    )
    return embeddings

In [27]:
embeddings = download_huggingface_embeddings()

/tmp/ipython-input-3391430880.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [28]:
query_result = embeddings.embed_documents(["Hà Nội là thủ đô của Việt Nam", "Đà Nẵng là thành phố du lịch"])
print("Chiều của vector 1:", len(query_result[0])) #- > thấy là dù cho vui_tính có dấu _ SBERT này vẫn xử lý tốt 768 embeddings đúng

Chiều của vector 1: 768


In [33]:
#Load .env co san len
from dotenv import load_dotenv
load_dotenv()

True